In [8]:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, \
    LoadTask, SaveTask, EOExecutor, ExtractBandsTask, MergeFeatureTask
from eolearn.io.processing_api import SentinelHubInputTask
from eolearn.core import LoadTask, SaveToDisk, AddFeature, RemoveFeature
from eolearn.visualization import EOPatchVisualization
from eolearn.mask import AddValidDataMaskTask
from shapely.ops import cascaded_union
from shapely.geometry import Polygon

from eolearn.features import SimpleFilterTask, NormalizedDifferenceIndexTask

from eolearn.geometry import VectorToRaster
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentinelhub as sh
from sentinelhub import UtmZoneSplitter, BBox, CRS, DataCollection

### Configuration of SentinelHub

In [2]:
config = sh.SHConfig()
config.sh_client_id= r'9bd0a46d-3d5b-4dc0-98b5-546b3635f9f3'
config.sh_client_secret = r'~)x%O:RiSc|F5i+SIL}^fZUlWOa.;E^{_:&!J6@:'
config.save()

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### File preparation

In [230]:
path = r'F:\geospatial_datasets\LUCAS_2018_Copernicus\LUCAS_2018_Copernicus'
pt = gpd.read_file(os.path.join(r'F:\geospatial_datasets\portugal_shapefile',r'PRT_adm1.shp'))
gdf = gpd.read_file(os.path.join(path,"LUCAS_2018_Copernicus_polygons.shp"))
csv = pd.read_csv(os.path.join(path,"LUCAS_2018_Copernicus_attributes.csv"))
gdf = gdf.to_crs(sh.CRS.WGS84.pyproj_crs())

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Create Portugal shapefile

In [231]:
portugal_csv = csv.loc[csv["NUTS0"] == "PT"]
gdf.POINT_ID = gdf.POINT_ID.astype(int)
portugal_gdf = portugal_csv.merge(gdf, on='POINT_ID', how='inner')
portugal_gdf.crs = gdf.crs
portugal_gdf = gpd.GeoDataFrame(portugal_gdf)
portugal_gdf = portugal_gdf.to_crs(sh.CRS.WGS84.pyproj_crs())
portugal_bounds = gpd.GeoSeries(cascaded_union(pt[pt.TYPE_1 == "Distrito"].geometry.to_list()))

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Subdivide portugal in smaller bbox

In [232]:

bbox_splitter = sh.BBoxSplitter(portugal_bounds.geometry.to_list(),sh.CRS.WGS84,(200,300))

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [233]:
geometry = [Polygon(bbox.get_polygon()) for bbox in bbox_splitter.get_bbox_list()]
gdf_bbox = gpd.GeoDataFrame(crs=portugal_gdf.crs, geometry=geometry)

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Select only bounding boxes that have relevant data

In [234]:
bbox_with_data = []
bbox_with_data = [gdf_bbox[gdf_bbox.geometry.contains(portugal_gdf.geometry[i])] for i,_ in enumerate(portugal_gdf.geometry)]
bbox_with_data = np.concatenate(bbox_with_data).ravel()
bbox_with_data = gpd.GeoDataFrame(geometry=bbox_with_data)
bbox_with_data.crs = portugal_gdf.crs

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Plot the created graphs

In [235]:
fig, ax = plt.subplots(figsize=(400, 400))

gdf_bbox.plot(ax=ax,facecolor='w',edgecolor='r',alpha=0.4)

portugal_bounds.plot(ax=ax, facecolor='w',edgecolor='b',alpha=0.5)

portugal_gdf.plot(ax=ax, edgecolor='b')

bbox_with_data.plot(ax=ax, edgecolor='green', facecolor='green')

ax.set_title('Bboxes', fontsize=25)

plt.axis('off');
plt.savefig('touches.png')

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Create and prepare EOTasks for the processing the data

In [236]:
class FilterVectorToRaster(EOTask):
    
    def __init__(self,raster_feature,values,raster_resolution):
        #this cant be here still, must add the args to the class and call vector2rastertask in the executemethod
        self.featureType = raster_feature
        self.values = values
        self.raster_resolution = raster_resolution
        
    def execute(self,eopatch,dataset):
        vector = dataset[dataset.geometry.intersects(eopatch.bbox.geometry)]
        print(type(vector))
        for i,vec in vector.iterrows():
            feature_name = self.featureType[-1]+"_"+str(vec.POINT_ID)
            feature = (self.featureType[0], feature_name)
            self.vector2rastertask = VectorToRaster(vector,raster_feature=feature,values=self.values,raster_resolution=self.raster_resolution)
            eopatch = self.vector2rastertask.execute(eopatch)
        return eopatch

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [237]:
def get_time_interval(middle_date, number_of_days):
    """
        Gets the time interval surrounding the middle date separated by slashes
    Args:
        middle_date: A string containing the date which will be included in the timerange
        number_of_days: The number of days counting from the `middle_date` that will correspond to the min and max date
        
    Returns:
        A list with the ´number_of_days´ before and after of the ´middle_date´
        
    Example:
        >>> get_time_interval("15/09/1998", 3)
        >>> ['1998-09-12', '1998-09-18']
        
    """
    point_date = datetime.datetime.strptime(middle_date, '%d/%m/%y')
    days_before = point_date - datetime.timedelta(days=number_of_days)
    days_after = point_date + datetime.timedelta(days=number_of_days)
    return [days_before.strftime('%Y-%m-%d'), days_after.strftime('%Y-%m-%d')]

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [238]:
add_data = SentinelHubInputTask(
    bands_feature=(FeatureType.DATA, 'BANDS'),
    resolution=10,
    maxcc=0.8,
    time_difference=datetime.timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L1C,
    max_threads=5,
    config=config,
)
path_out = r'F:\geospatial_datasets\eopatches'
if not os.path.isdir(path_out):
    os.makedirs(path_out)
save = SaveTask(path_out, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)
add_lucas = AddFeature((FeatureType.META_INFO,"LUCAS_DATA"))
add_lucas_raster = FilterVectorToRaster(raster_feature=(FeatureType.MASK_TIMELESS,"GROUND_TRUTH"),values=1,raster_resolution=10)

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [239]:
workflow = LinearWorkflow(add_data,add_lucas,add_lucas_raster,save)

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [240]:
# Execute the workflow

# define additional parameters of the workflow
execution_args = []
for idx, bbox in enumerate(bbox_with_data.geometry.to_list()[:5]):
    lucas_points_intersection = portugal_gdf[portugal_gdf.intersects(bbox)]
    time_interval = []
    for point in lucas_points_intersection.SURVEY_DATE:
        time_interval = (get_time_interval(point,5))
    execution_args.append({
        add_data:{'bbox': BBox(bbox,crs=portugal_gdf.crs), 'time_interval': time_interval},
        add_lucas:{'data': lucas_points_intersection},
        add_lucas_raster:{'dataset':lucas_points_intersection},
        save: {'eopatch_folder': f'eopatch_{idx}'}
    })
executor = EOExecutor(workflow, execution_args, save_logs=True)
executor.run(workers=5, multiprocess=False)

executor.make_report()

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/5 [00:00<?, ?it/s]

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


### Missing steps:
    [x] Get shapefile of portugal and recalculate bboxes 
    [ ] Get the images regarding the week before and after of the ground truth timestamp
    [X] Redo the second workflow that rasterizes the bboxes of lucas copernicus
    [X] Ensure the WGS TO sentinelhub.CRS.WGS84 
    [ ] Execute the 

In [241]:
patch = EOPatch.load(r'F:\geospatial_datasets\eopatches\eopatch_0',lazy_loading=False)

C:\Users\FEEL\anaconda3\envs\Geospatial\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
